In [1]:
import json
import boto3
import time
from anthropic import Anthropic 
from botocore.exceptions import ClientError
from chunkipy import TextChunker, TokenEstimator
from transformers import AutoTokenizer #, BertTokenizer, pipeline

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BertTokenEstimator(TokenEstimator):
    def __init__(self):
        self.bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    def estimate_tokens(self, text):
        return len(self.bert_tokenizer.encode(text))

In [3]:
# Initialize BertEstimator
bert_token_estimator = BertTokenEstimator()

In [4]:
REGION = "us-east-1"

In [5]:
client_bedrock = boto3.client("bedrock-runtime", REGION)

In [6]:
txt = """Mission Two: The Wrecking Crew
Matt Helm series Reviews by Ujjwal Dey

Well the first one is beaten to grit and grim by this second one in the series. A little longer at 176 pages, still an easy read for my weekend; this one brings Helm into a whole new line of action in espionage business.

Now Matt Helm has had his refresher course in the covert groups training - the American Mordgruppe - The Wrecking Crew - an unknown, unspoken elite group of operatives who generally work alone to do what armies and clouts of bureaucrats can't achieve. The man is just right for the job. The trainers believe he is in no shape to be an operative and certainly past his prime. His bad new resume was certain to get him killed on a field mission. Mac agrees - he is just the man for this job.

Matt Helm now has to play dumb; to act like a clumsy ancient World War trooper who can't call the shots in this peacetime covert warfare. As a photographer for an American magazine he lands up in Artic Europe to shoot innocent bland photos of mines for a girl who could be a double-agent or simply a fool in this foolish game. The girl in question has survived a bullet meant for her journalist husband who had the gall to write a tale describing a Russian agent - The Man No one Knows. Of course this deadly Russian operative has no sympathy for such breakthrough journalism, and now the supposed widow is carrying out her husband's journalistic inclinations.

There is more than meets the eye and pretty women are lethal in more ways than one. Helm's contact in Sweden is shot dead in the face, double-crossed by her evil mysterious agent. Helm has to contend with getting bruised and bashed around to prove himself harmless to a variety of operatives - biding his time to get his orders.

Yes, the men in Washington call off lethal action - no Government ordered assassination during peacetime - but that is not a restriction upon Helm's enemies. Matt Helm goes through unraveling intricacies in the players' cards, as a poker player who has to display ignorance of any known card game. As people show up dead around him, things come to light and when the final game is afoot - Matt has the aces up his sleeve to vindicate Mac's faith in him.

The violence is as bad if not worse than the first book - which is a good thing in any gritty espionage thriller. The brief reflections on the first book events such as him carving up an old lady friend and his separation from his wife also come up very much accurately into the new plot. His handiness without a gun is seen very well in this story as essentially he has to go out there unarmed to convincingly play the role of an American photographer, even if the cover doesn't fool his targets.

The geographical descriptions and accuracy in detail is wonderful and you can imagine yourself tracing his trail across the mountains and into wilderness in the Arctic. He has to "make the touch" - Group M speak for killing the target - similar to what mafia would say "making a hit". But he has to be patient enough to identify the mysterious Russian spy, wait for the go ahead from his Boss, and then make sure he does it cleanly - being in a friendly country during peacetime.

Putting up a classy display of ineptness, we also get to read about all that he could have done as a master agent but doesn't to keep himself useful to the Russian agent - he is able to prove himself harmless on more than one occasion until finally its time for a showdown.

Cars, guns, women's choice of clothes, all again feature in this sequel in Matt Helm's ponderings. There are women he trusts and they assuredly betray him and Helm is not one to be heartbroken or sentimental - he goes about his business with determination and calculation - even surprising his own Government's other operatives (of other departments) - who fall for his "clumsy" act. At the end Helm proves himself to be as cunning and ruthless as his Russian rival. The climax action with its cold-blooded moves sees Helm make his touch and save a damsel from distress as well. The last chapter adds more to Helm's personality and legend. His un-emotive demeanour at what could have been a tragic romantic scene ensures he is the man with a job he is good at.

If you thought gadgets and expensive machinery with latest guns was the way an agent wins a war - you have watched too many James Bond movies. This book's account shows us in a believable and clinical clarity how a secret agent would go through with his mission in a foreign country. Matt Helm is no great fist-fighter but he knows how to fight and here we see him use more of the matter between his ears in contrast to the trigger in enemy hands.

Extract: When you act like a nice guy, everyone examines your motives with a microscope. When you act like a conscienceless louse, they generally take you at face value."""

In [7]:
token_count = bert_token_estimator.estimate_tokens(txt)
print(token_count)

Token indices sequence length is longer than the specified maximum sequence length for this model (1040 > 512). Running this sequence through the model will result in indexing errors


1040


In [8]:
# Define model ID and prompt
model_id = 'anthropic.claude-v2:1'
system_prompt = "All your output has to be grammatically correct in a single paragraph"
user_prompt = f"""Write a summary of the following review:\n{txt}"""
prompt = f"""System: {system_prompt}\n\nHuman: {user_prompt}\n\nAssistant:"""

message_json = {
    "role": "user",
    "content": [
       {
           "type": "text",
           "text": user_prompt
       }
    ]
}

body = json.dumps({
    "max_tokens": 2048,
    "temperature": 0.5,
    "messages": [ message_json ],
    "anthropic_version": "bedrock-2023-05-31"
})


In [9]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response = client_bedrock.invoke_model(modelId=model_id, body=body)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

14:59:02
14:59:17


In [10]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 15.0


In [11]:
response_body = json.loads(response["body"].read())

In [12]:
print(response_body["content"][0]["text"])

Here is a summary of the review:

This is a review of the second book in the Matt Helm spy thriller series, "The Wrecking Crew." The review praises this sequel as being even better than the first book, with more action and violence. The plot involves Helm going on a mission to the Arctic as a photographer, where he must identify and eliminate a mysterious Russian agent. Helm acts clumsy and harmless as part of his cover, while navigating intrigues with potential double agents. The review highlights the accurate details, locations, action sequences, and cold-blooded nature of Helm's spy craft. It notes Helm's detachment and lack of emotion even in romantic situations. The review states this book shows the cerebral tactics and ruthlessness needed in real-world espionage, contrasting it favorably to flashy James Bond-style stories. It concludes by saying Helm proves as cunning as his Russian rival target in the climactic confrontation.


# Using Claude 3 Haiku

In [13]:
# Define model ID and prompt
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
system_prompt = "All your output has to be grammatically correct in a single paragraph"
user_prompt = f"""Write a summary of the following review:\n{txt}"""
prompt = f"""System: {system_prompt}\n\nHuman: {user_prompt}\n\nAssistant:"""

message_json = {
    "role": "user",
    "content": [
       {
           "type": "text",
           "text": user_prompt
       }
    ]
}

body = json.dumps({
    "max_tokens": 2048,
    "temperature": 0.5,
    "messages": [ message_json ],
    "anthropic_version": "bedrock-2023-05-31"
})


In [14]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response = client_bedrock.invoke_model(modelId=model_id, body=body)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

14:59:45
14:59:49


In [15]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 4.0


In [16]:
response_body = json.loads(response["body"].read())

In [17]:
print(response_body["content"][0]["text"])

Here is a summary of the review:

The review discusses the second book in the Matt Helm series, "The Wrecking Crew". It notes that this book is longer than the first at 176 pages, but still an easy weekend read. 

The review explains that in this book, Matt Helm has gone through training with a covert American group called "The Wrecking Crew", an elite group of operatives. Despite being seen as past his prime, Helm is selected for a mission where he has to play the role of a clumsy, aging photographer in order to gather intelligence.

The review highlights Helm's cunning and ruthlessness as he navigates the intricacies of the mission, which involves a mysterious Russian agent and various double-agents and operatives. Helm has to bide his time and play the part of the harmless photographer, all while gathering the information he needs to complete his objective.

The review praises the vivid geographical descriptions and Helm's ability to use his mind rather than just brute force to over